# Whisper LoRA Fine-Tuning (Kaggle, Custom Loop, No Trainer)

This notebook fine-tunes a **Whisper model with LoRA (PEFT)** on your Tunisian medical ASR dataset on Kaggle.

Key design choices:
- Uses **LoRA** (only a small % of weights are trainable → much lighter).
- Uses a **manual training loop** (no `Seq2SeqTrainer`, so no double-backward bug).
- Designed for **Kaggle T4 GPU** (14–16 GB VRAM).
- Truncates audio to a max duration to keep memory usage controlled.

You can switch the model between:
- `openai/whisper-tiny` (safest)
- `openai/whisper-base` (recommended)
- `openai/whisper-small` (heavier, may be too big on Kaggle)


In [2]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [3]:
print(torch.cuda.device_count())

1


## 1. Install dependencies

In [4]:
!ls /kaggle/input

data_processed	test.jsonl  train.jsonl  valid.jsonl


In [5]:
!ls /kaggle/input/tunisian-whisper-asr-dataset/data_processed

ls: cannot access '/kaggle/input/tunisian-whisper-asr-dataset/data_processed': No such file or directory


In [6]:
!pip install -q datasets transformers peft evaluate jiwer soundfile librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 60.5 MB/s eta 0:00:00:00:01


## 2. Imports & basic setup

In [7]:
import os
import json
import math
import time
import random

import numpy as np
import pandas as pd
import librosa


from datasets import Dataset
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    get_linear_schedule_with_warmup,
)
from peft import LoraConfig, get_peft_model
import evaluate

print("Torch version:", torch.__version__)
import transformers
print("Transformers version:", transformers.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Torch version: 2.8.0+cu126
Transformers version: 4.57.1
Using device: cuda


## 3. Load JSONL dataset and fix audio paths

We assume the Kaggle dataset structure:
```text
/kaggle/input/tunisian-whisper-asr-dataset/
    data_processed/
        data_processed/
            segments/
                train/train_000000.wav
                dev/dev_000000.wav
                test/test_000000.wav
    train.jsonl
    valid.jsonl
    test.jsonl
```

The JSONL files contain **relative Windows-style paths** like:
- `data_processed\\segments\\train\\train_000000.wav`
- `data_processed\\segments\\dev\\train_000000.wav` (but files are actually named `dev_000000.wav`)
- `data_processed\\segments\\test\\train_000000.wav` (but files are actually named `test_000000.wav`)

We:
- Normalize `\\` → `/`
- Prepend the Kaggle prefix `/kaggle/input/tunisian-whisper-asr-dataset/data_processed/`
- Fix wrong `train_` prefix in the dev and test splits
- Skip entries whose audio file does not exist


In [8]:
KAGGLE_PREFIX = "/kaggle/input/data_processed/"

def load_jsonl_train(jsonl_path: str) -> Dataset:
    data = []
    missing = 0
    loaded = 0
    print(f"\n🔄 Loading TRAIN from: {jsonl_path}")
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            item = json.loads(line)
            raw_path = item["audio"].replace("\\", "/")
            kaggle_path = KAGGLE_PREFIX + raw_path
            if os.path.exists(kaggle_path):
                data.append({"audio": kaggle_path, "text": item["text"]})
                loaded += 1
            else:
                missing += 1
                print(f"⚠️ [train] Missing file, skipped: {kaggle_path}")
    print(f"📊 [train] Loaded: {loaded} | Missing: {missing}\n")
    if not data:
        raise RuntimeError("No valid TRAIN samples found.")
    return Dataset.from_pandas(pd.DataFrame(data))

def load_jsonl_dev(jsonl_path: str) -> Dataset:
    data = []
    missing = 0
    loaded = 0
    print(f"\n🔄 Loading DEV from: {jsonl_path}")
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            item = json.loads(line)
            raw_path = item["audio"].replace("\\", "/")
            if "segments/dev/" in raw_path and "train_" in raw_path:
                raw_path = raw_path.replace("train_", "dev_")
            kaggle_path = KAGGLE_PREFIX + raw_path
            if os.path.exists(kaggle_path):
                data.append({"audio": kaggle_path, "text": item["text"]})
                loaded += 1
            else:
                missing += 1
                print(f"⚠️ [dev] Missing file, skipped: {kaggle_path}")
    print(f"📊 [dev] Loaded: {loaded} | Missing: {missing}\n")
    if not data:
        raise RuntimeError("No valid DEV samples found.")
    return Dataset.from_pandas(pd.DataFrame(data))

def load_jsonl_test(jsonl_path: str) -> Dataset:
    data = []
    missing = 0
    loaded = 0
    print(f"\n🔄 Loading TEST from: {jsonl_path}")
    with open(jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            item = json.loads(line)
            raw_path = item["audio"].replace("\\", "/")
            if "segments/test/" in raw_path and "train_" in raw_path:
                raw_path = raw_path.replace("train_", "test_")
            kaggle_path = KAGGLE_PREFIX + raw_path
            if os.path.exists(kaggle_path):
                data.append({"audio": kaggle_path, "text": item["text"]})
                loaded += 1
            else:
                missing += 1
                print(f"⚠️ [test] Missing file, skipped: {kaggle_path}")
    print(f"📊 [test] Loaded: {loaded} | Missing: {missing}\n")
    if not data:
        raise RuntimeError("No valid TEST samples found.")
    return Dataset.from_pandas(pd.DataFrame(data))


In [9]:
train_jsonl_path = "/kaggle/input/train.jsonl"
valid_jsonl_path = "/kaggle/input/valid.jsonl"
test_jsonl_path  = "/kaggle/input/test.jsonl"

train_ds = load_jsonl_train(train_jsonl_path)
valid_ds = load_jsonl_dev(valid_jsonl_path)
test_ds  = load_jsonl_test(test_jsonl_path)

print(train_ds)
print(valid_ds)
print(test_ds)


🔄 Loading TRAIN from: /kaggle/input/train.jsonl
📊 [train] Loaded: 15703 | Missing: 0


🔄 Loading DEV from: /kaggle/input/valid.jsonl
📊 [dev] Loaded: 731 | Missing: 0


🔄 Loading TEST from: /kaggle/input/test.jsonl
📊 [test] Loaded: 842 | Missing: 0

Dataset({
    features: ['audio', 'text'],
    num_rows: 15703
})
Dataset({
    features: ['audio', 'text'],
    num_rows: 731
})
Dataset({
    features: ['audio', 'text'],
    num_rows: 842
})


## 4. (Optional) Use only a subset (e.g. 10%) for faster experiments

In [10]:
use_fraction = 0.1  # 10% of each split

def sample_fraction(ds, frac, seed=42):
    if frac >= 1.0:
        return ds
    n = len(ds)
    k = max(1, int(n * frac))
    return ds.shuffle(seed=seed).select(range(k))

train_ds_small = sample_fraction(train_ds, use_fraction)
valid_ds_small = sample_fraction(valid_ds, use_fraction)
test_ds_small  = sample_fraction(test_ds,  use_fraction)

print("Train subset:", len(train_ds_small))
print("Valid subset:", len(valid_ds_small))
print("Test subset:",  len(test_ds_small))

Train subset: 1570
Valid subset: 73
Test subset: 84


## 5. Load Whisper model, processor & apply LoRA

In [11]:
# Choose your base model
model_name = "openai/whisper-base"  # or "openai/whisper-tiny"

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="ar", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name, language="ar", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
)

model = get_peft_model(model, lora_config)
model.to(device)
model.print_trainable_parameters()

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

trainable params: 589,824 || all params: 73,183,744 || trainable%: 0.8059


## 6. Preprocess audio & text

In [12]:
MAX_DURATION_S = 20
SAMPLE_RATE = 16000

def prepare_example(batch):
    audio_path = batch["audio"]
    audio, sr = librosa.load(audio_path, sr=SAMPLE_RATE)
    max_samples = MAX_DURATION_S * SAMPLE_RATE
    if len(audio) > max_samples:
        audio = audio[:max_samples]

    batch["input_features"] = feature_extractor(
        audio, sampling_rate=SAMPLE_RATE
    ).input_features[0]
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch


In [13]:
train_proc = train_ds_small.map(prepare_example, remove_columns=["audio", "text"])
valid_proc = valid_ds_small.map(prepare_example, remove_columns=["audio", "text"])
test_proc  = test_ds_small.map(prepare_example,  remove_columns=["audio", "text"])

train_proc[0]

Map:   0%|          | 0/1570 [00:00<?, ? examples/s]

Map:   0%|          | 0/73 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

{'input_features': [[0.6020662784576416,
   0.2054188847541809,
   -0.04294288158416748,
   -0.05440318584442139,
   0.04740297794342041,
   0.022235751152038574,
   -0.11919498443603516,
   0.13073068857192993,
   0.1469419002532959,
   -0.09347832202911377,
   0.061231255531311035,
   0.2301521897315979,
   -0.03562521934509277,
   0.08268988132476807,
   0.07985061407089233,
   0.22203397750854492,
   0.14616411924362183,
   0.03747427463531494,
   -0.06655299663543701,
   -0.31040453910827637,
   -0.27668893337249756,
   0.10426163673400879,
   -0.01721954345703125,
   0.1335355043411255,
   -0.03267371654510498,
   0.07772636413574219,
   -0.04008591175079346,
   0.05404931306838989,
   -0.39047670364379883,
   -0.10985052585601807,
   -0.23235809803009033,
   -0.18630719184875488,
   -0.2604331970214844,
   0.29614120721817017,
   0.29771894216537476,
   0.2856781482696533,
   0.28883951902389526,
   0.24009281396865845,
   -0.040958285331726074,
   0.20198982954025269,
   0.1090

## 9. Training loop (LoRA, custom, AMP)

In [14]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def whisper_data_collator(features):
    input_features = [f["input_features"] for f in features]
    labels = [f["labels"] for f in features]

    # Pad audio features
    batch = processor.feature_extractor.pad(
        {"input_features": input_features},
        return_tensors="pt"
    )

    # Pad labels
    labels_batch = processor.tokenizer.pad(
        {"input_ids": labels},
        return_tensors="pt"
    )
    labels = labels_batch["input_ids"]
    labels[labels == processor.tokenizer.pad_token_id] = -100

    batch["labels"] = labels
    return batch

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

class WhisperTrainer(Seq2SeqTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # remove any text-model keys if they exist
        inputs.pop("input_ids", None)
        inputs.pop("attention_mask", None)
        inputs.pop("decoder_input_ids", None)
        inputs.pop("decoder_attention_mask", None)

        # 🔑 IMPORTANT: call the underlying whisper model, not the PEFT wrapper forward
        base = model.get_base_model()  # works for PEFT models

        outputs = base(
            input_features=inputs["input_features"],
            labels=inputs["labels"],
        )

        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-lora",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    predict_with_generate=True,
    generation_max_length=225,
    remove_unused_columns=False,
)

trainer = WhisperTrainer(
    model=model,
    args=training_args,
    train_dataset=train_proc,
    eval_dataset=valid_proc,
    data_collator=whisper_data_collator,
)

trainer.train()


You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.937300,4.149001
2,3.771600,4.021814
3,3.716800,3.989339


TrainOutput(global_step=591, training_loss=3.914504127050616, metrics={'train_runtime': 1142.6565, 'train_samples_per_second': 4.122, 'train_steps_per_second': 0.517, 'total_flos': 3.094910042112e+17, 'train_loss': 3.914504127050616, 'epoch': 3.0})

In [24]:
import accelerate
print(accelerate.__version__)

1.11.0


In [25]:
print(type(model))
print(type(model.get_base_model()))

<class 'peft.peft_model.PeftModelForSeq2SeqLM'>
<class 'transformers.models.whisper.modeling_whisper.WhisperForConditionalGeneration'>


## 10. Save fine-tuned LoRA model

In [26]:
output_dir = "./whisper-base-tunisian-lora"
os.makedirs(output_dir, exist_ok=True)
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)
print(f"Saved LoRA model and processor to {output_dir}")

Saved LoRA model and processor to ./whisper-base-tunisian-lora


## 11. Quick inference example

In [29]:
def transcribe(idx):
    ex = test_proc[idx]
    feats = torch.tensor(ex["input_features"]).unsqueeze(0).to(model.device)
    with torch.no_grad():
        ids = model.get_base_model().generate(feats, max_new_tokens=128)
    return processor.batch_decode(ids, skip_special_tokens=True)[0]

print(transcribe(0))


والكي تحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في الناس ويحبوا في النا


In [30]:
import evaluate
import torch
from tqdm import tqdm

wer_metric = evaluate.load("wer")

def evaluate_wer(model, dataset, processor, max_samples=None):
    model.eval()
    predictions = []
    references = []

    for i, ex in enumerate(tqdm(dataset)):
        if max_samples is not None and i >= max_samples:
            break

        input_features = torch.tensor(ex["input_features"]).unsqueeze(0).to(model.device)

        with torch.no_grad():
            generated_ids = model.get_base_model().generate(
                input_features,
                max_new_tokens=128
            )

        pred_text = processor.batch_decode(
            generated_ids, skip_special_tokens=True
        )[0]

        label_ids = ex["labels"]
        label_ids = [t for t in label_ids if t != -100]

        ref_text = processor.decode(label_ids, skip_special_tokens=True)

        predictions.append(pred_text)
        references.append(ref_text)

    wer = wer_metric.compute(predictions=predictions, references=references)
    return wer

In [31]:
val_wer = evaluate_wer(
    model=model,
    dataset=valid_proc,
    processor=processor,
    max_samples=100  # start small, then remove
)

print(f"Validation WER: {val_wer:.4f}")

100%|██████████| 73/73 [01:40<00:00,  1.38s/it]

Validation WER: 2.4312


In [32]:
test_wer = evaluate_wer(
    model=model,
    dataset=test_proc,
    processor=processor,
    max_samples=100
)

print(f"Test WER: {test_wer:.4f}")

100%|██████████| 84/84 [01:52<00:00,  1.34s/it]

Test WER: 2.7437
